In [30]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pickle
import re
from umap import UMAP
from hdbscan import HDBSCAN
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel
from collections.abc import Iterable
from transformers import AutoTokenizer, AutoModel
from bertopic import BERTopic
import torch
from collections import deque
from bertopic.representation import KeyBERTInspired


In [2]:
# 파일 불러오기
with open('../../data/bert_df.pkl', 'rb') as f:
    df = pickle.load(f)


In [3]:
# 파일 불러오기
with open('../../data/bert_src_df.pkl', 'rb') as f:
    bert_src_df = pickle.load(f)


In [4]:
bert_src_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5223203 entries, 0 to 5223202
Data columns (total 5 columns):
 #   Column          Dtype         
---  ------          -----         
 0   q_id            int64         
 1   a_id            float64       
 2   q_creationdate  datetime64[ns]
 3   tags            object        
 4   body            object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 199.2+ MB


In [5]:
cond1 = bert_src_df['q_creationdate']<='2022-11-30'
cond2 = bert_src_df['q_creationdate']>='2021-11-30'
cond3 = bert_src_df['a_id'].isna()
cond4 = bert_src_df['tags'].str.contains('python')
cond5 = bert_src_df['q_creationdate']>='2022-12-01'

In [6]:
bert_src_bf = bert_src_df.loc[cond1 & cond2 & cond3 & cond4, :]
bert_src_af = bert_src_df.loc[cond3 & cond4 & cond5, :]

In [7]:
bert_src_af

,q_id,a_id,q_creationdate,tags,body
38,77077227,NaN,2023-09-10 17:31:31.600,<python><opengl><pyqt5><pyopengl>,<p>I want to display 3d object on top of PyQT5...
65,77060888,NaN,2023-09-07 15:17:24.040,<python><django><postgresql><sleep><pytest-dja...,<p>I need to make a pause in my Django project...
73,77052025,NaN,2023-09-06 12:43:17.777,<python>,<p>I am trying to get a list of all compartmen...
107,76937634,NaN,2023-08-20 01:18:19.203,<python><django><asynchronous><celery><telegram>,<p>I'm trying to make a parser like a web appl...
115,76934523,NaN,2023-08-19 09:57:53.073,<python><user-interface><pyqt5>,"<p>guys I have a question, i have a code and I..."
...,...,...,...,...,...
5223137,77545627,NaN,2023-11-24 20:54:56.123,<python><huggingface><language-translation>,<h2>Setup</h2>\n<p>I've created a HF Inference...
5223163,77442875,NaN,2023-11-08 04:40:57.380,<python><python-imaging-library><height><width>,<p>We were given a image(The Traffic stop sign...
5223179,77374676,NaN,2023-10-27 13:47:11.517,<python><tensorflow><keras>,<p>I am having an issue I don't understand how...
5223196,77336446,NaN,2023-10-21 14:33:34.623,<python><animation><sequence><vtk>,<p>I have succesfully managed to make a 3d ima...


In [8]:
def cleanhtml(raw_html):
  # 1.Source code in python language is hard to understand, so replace all the <code> tag first
  cleantext_1 = re.findall(r'(?<=\<code>)(.*?)(?=<\/code>)', raw_html.replace('\n', '_**_'))
  cleantext_1 = [x.replace('_**_', '\n') for x in cleantext_1]
  # 2. replace html tags
  # <p>
  tag_re = re.compile('<.*?>')
  cleantext_2 = [re.sub(tag_re, '', x) for x in cleantext_1]
  return cleantext_2

In [9]:
# apply the function, cleanhtml to the question and body text
bert_src_bf.loc[:, 'q_prep_text'] = bert_src_bf['body'].apply(cleanhtml)
bert_src_af.loc[:, 'q_prep_text'] = bert_src_af['body'].apply(cleanhtml)

In [10]:
bert_src_af

,q_id,a_id,q_creationdate,tags,body,q_prep_text
38,77077227,NaN,2023-09-10 17:31:31.600,<python><opengl><pyqt5><pyopengl>,<p>I want to display 3d object on top of PyQT5...,[import sys\nfrom OpenGL.GL import *\nfrom Ope...
65,77060888,NaN,2023-09-07 15:17:24.040,<python><django><postgresql><sleep><pytest-dja...,<p>I need to make a pause in my Django project...,"[from pytest import mark, fixture, raises\nfro..."
73,77052025,NaN,2023-09-06 12:43:17.777,<python>,<p>I am trying to get a list of all compartmen...,[import oci\n\nconfig = oci.config.from_file()...
107,76937634,NaN,2023-08-20 01:18:19.203,<python><django><asynchronous><celery><telegram>,<p>I'm trying to make a parser like a web appl...,[from telethon.sync import TelegramClient\nimp...
115,76934523,NaN,2023-08-19 09:57:53.073,<python><user-interface><pyqt5>,"<p>guys I have a question, i have a code and I...",[1\.when moving the icon instances the X and Y...
...,...,...,...,...,...,...
5223137,77545627,NaN,2023-11-24 20:54:56.123,<python><huggingface><language-translation>,<h2>Setup</h2>\n<p>I've created a HF Inference...,"[request, concurrent, def translate_text(text)..."
5223163,77442875,NaN,2023-11-08 04:40:57.380,<python><python-imaging-library><height><width>,<p>We were given a image(The Traffic stop sign...,[ def fix_middle(picture):\n picture...
5223179,77374676,NaN,2023-10-27 13:47:11.517,<python><tensorflow><keras>,<p>I am having an issue I don't understand how...,"[ def create_teacher_model(img_size, model_..."
5223196,77336446,NaN,2023-10-21 14:33:34.623,<python><animation><sequence><vtk>,<p>I have succesfully managed to make a 3d ima...,[import vtkmodules.all as vtk\n\n#Create a ren...


In [11]:
bert_src_bf = bert_src_bf.reset_index(drop=True)
bert_src_af = bert_src_af.reset_index(drop=True)

In [12]:
bert_src_bf = bert_src_bf[['q_id', 'q_prep_text']].apply(pd.Series.explode)
bert_src_af = bert_src_af[['q_id', 'q_prep_text']].apply(pd.Series.explode)

In [13]:
bert_src_bf.dropna(inplace=True)
bert_src_af.dropna(inplace=True)

In [14]:
# PREPROCESSING FOR CODE SCRIPT
def preprocess_script(script):
    new_script = deque()
    old_script = script.split('\n')
    for line in old_script:
        if line.lstrip().startswith('#'): # 주석으로 시작되는 행 skip
            continue
        line = line.rstrip()
        if '#' in line:
            line = line[:line.index('#')] # 주석 전까지 코드만 저장
        line = line.replace('\n','') # 개행 문자를 모두 삭제함
        line = line.replace('    ','\t') # 공백 4칸을 tab으로 변환
        
        if line == '': # 전처리 후 빈 라인은 skip
            continue
        
        new_script.append(line)

        
    new_script = '\n'.join(new_script) # 개행 문자로 합침
    new_script = re.sub('("""[\w\W]*?""")', '<str>', new_script)
    new_script = re.sub("('''[\w\W]*?''')", '<str>', new_script)
    new_script = re.sub('/^(http?|https?):\/\/([a-z0-9-]+\.)+[a-z0-9]{2,4}.*$/', '', new_script)
    
    return new_script


In [15]:
bert_src_af

,q_id,q_prep_text
0,77077227,import sys\nfrom OpenGL.GL import *\nfrom Open...
1,77060888,"from pytest import mark, fixture, raises\nfrom..."
1,77060888,========================================= test...
2,77052025,import oci\n\nconfig = oci.config.from_file()\...
3,76937634,from telethon.sync import TelegramClient\nimpo...
...,...,...
63443,77336446,import time\nimport vtkmodules.all\n\nclass vt...
63444,77336843,apiVersion: networking.k8s.io/v1\nkind: Ingres...
63444,77336843,apiVersion: networking.k8s.io/v1\nkind: Ingres...
63444,77336843,apiVersion: v1\nkind: Service\nmetadata:\n na...


In [16]:
bert_src_bf['q_prep_text_non'] = bert_src_bf['q_prep_text'].apply(preprocess_script)
bert_src_af['q_prep_text_non'] = bert_src_af['q_prep_text'].apply(preprocess_script)

In [17]:
src = bert_src_af['q_prep_text_non'].tolist()
# df['sentiments'].values.tolist()


In [18]:
src

["import sys\nfrom OpenGL.GL import *\nfrom OpenGL.GLU import *\nfrom PyQt5 import QtGui\nfrom PyQt5.QtOpenGL import *\nfrom PyQt5 import QtCore, QtWidgets, QtOpenGL\nclass Ui_MainWindow(QtWidgets.QWidget):\n\tdef __init__(self, parent=None):\n\t\tsuper(Ui_MainWindow, self).__init__()\n\t\tself.widget = glWidget()\n\t\tself.button = QtWidgets.QPushButton('Test', self)\n\t\tmainLayout = QtWidgets.QGridLayout()\n\t\tmainLayout.addWidget(self.widget,0,0)\n\t\tmainLayout.addWidget(self.button,0,0)\n\t\tself.setLayout(mainLayout)\nclass glWidget(QGLWidget):\n\tdef __init__(self, parent=None):\n\t\tQGLWidget.__init__(self, parent)\n\t\tself.setMinimumSize(640, 480)\n\tdef paintGL(self):\n\t\tglClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)\n\t\tglLoadIdentity()\n\t\tglTranslatef(-5, 0.5, -6.0)\n\t\tglColor3f( 1.0, 1.5, 0.0 );\n\t\tglPolygonMode(GL_FRONT, GL_FILL);\n\t\tglBegin(GL_TRIANGLES)\n\t\tglVertex3f(2.0,-1.2,0.0)\n\t\tglVertex3f(2.6,0.0,0.0)\n\t\tglVertex3f(2.9,-1.2,0.0)\n\t\tglEnd(

In [19]:
print(type(src))
print(type(src[0]))
print(len(src))

<class 'list'>
<class 'str'>
173400


In [20]:
src = src[:1000]

In [21]:
# 데이터 로드
data = src

In [22]:
data = data

In [23]:
# CodeBERT 모델과 토크나이저 로드
model_name = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [24]:
# 임베딩 함수 정의
def embed_text(text):
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    return model_output.last_hidden_state[:, 0, :]

In [25]:
# 모든 텍스트에 대한 임베딩 계산
embeddings = torch.cat([embed_text(text) for text in data], dim=0).numpy()  # 텐서로 연결 후 numpy 배열로 변환

In [26]:
embeddings.shape

(1000, 768)

In [27]:
umap_model = UMAP(n_neighbors=10, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

In [28]:
hdbscan_model = HDBSCAN(min_cluster_size=20, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [31]:
representation_model = KeyBERTInspired()

In [32]:
# BERTopic 모델 초기화 및 훈련
topic_model = BERTopic( embedding_model=model,
                        umap_model=umap_model,
                        hdbscan_model=hdbscan_model,
                        representation_model=representation_model)  # 임베딩 모델 사용을 비활성화
topics, probabilities = topic_model.fit_transform(data, embeddings)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [33]:
# 결과 출력
print(topic_model.get_topic_info())  # 토픽 정보 출력

   Topic  Count                                             Name  \
0     -1     34              -1_bootstrap_button_href_stylesheet   
1      0    337                       0_python3_python_pygame_py   
2      1    275                                1_10_py_error_100   
3      2    241                   2_projectpoints_cv_pyenv_index   
4      3     85  3_uninitializederror_typeerror_to_dlpack_module   
5      4     28             4_script_02_script_01_python3_python   

                                      Representation  \
0  [bootstrap, button, href, stylesheet, onclick,...   
1  [python3, python, pygame, py, print, 00, 01, t...   
2  [10, py, error, 100, name, python, used, type,...   
3  [projectpoints, cv, pyenv, index, env, path, n...   
4  [uninitializederror, typeerror, to_dlpack, mod...   
5  [script_02, script_01, python3, python, py, gu...   

                                 Representative_Docs  
0  [&lt;?xml version=&quot;1.0&quot;?&gt;\n&lt;Al...  
1  [{&quot;8&quot;: 

In [34]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,34,-1_bootstrap_button_href_stylesheet,"[bootstrap, button, href, stylesheet, onclick,...",[&lt;?xml version=&quot;1.0&quot;?&gt;\n&lt;Al...
1,0,337,0_python3_python_pygame_py,"[python3, python, pygame, py, print, 00, 01, t...","[{&quot;8&quot;: &quot;huit&quot;, &quot;5&quo..."
2,1,275,1_10_py_error_100,"[10, py, error, 100, name, python, used, type,...",[{\n &quot;repoOwner&quot;: &quot;&lt;REPOSIT...
3,2,241,2_projectpoints_cv_pyenv_index,"[projectpoints, cv, pyenv, index, env, path, n...","[cv.projectPoints, cv.projectPoints, cv.projec..."
4,3,85,3_uninitializederror_typeerror_to_dlpack_module,"[uninitializederror, typeerror, to_dlpack, mod...","[no module named numpy, ModuleNotFoundError: N..."
5,4,28,4_script_02_script_01_python3_python,"[script_02, script_01, python3, python, py, gu...","[script_02.py, script_02.py, script_02.py]"


In [35]:
topic_model.get_topic(0)

[('python3', 0.35473585),
 ('python', 0.32156235),
 ('pygame', 0.3008075),
 ('py', 0.29041278),
 ('print', 0.26775756),
 ('00', 0.24752432),
 ('01', 0.23277721),
 ('tf', 0.22753753),
 ('20', 0.22458306),
 ('tk', 0.22380301)]

In [36]:
topic_model.get_topic(1)

[('10', 0.37775224),
 ('py', 0.32384515),
 ('error', 0.31744936),
 ('100', 0.28027898),
 ('name', 0.27083427),
 ('python', 0.26763445),
 ('used', 0.26676336),
 ('type', 0.26123512),
 ('encoded', 0.26109916),
 ('module', 0.2557831)]

In [37]:
topic_model.get_topic(6, full=True)

False

In [ ]:
# # Label the topics yourself
# topic_model.set_topic_labels({1: "Space Travel", 7: "Religion"})

# # or use one of the other topic representations, like KeyBERTInspired
# keybert_topic_labels = {topic: " | ".join(list(zip(*values))[0][:3]) for topic, values in topic_model.topic_aspects_["KeyBERT"].items()}
# topic_model.set_topic_labels(keybert_topic_labels)

# # or ChatGPT's labels
# chatgpt_topic_labels = {topic: " | ".join(list(zip(*values))[0]) for topic, values in topic_model.topic_aspects_["OpenAI"].items()}
# chatgpt_topic_labels[-1] = "Outlier Topic"
# topic_model.set_topic_labels(chatgpt_topic_labels)

In [ ]:
# topic_distr, _ = topic_model.approximate_distribution(data, window=8, stride=4)

In [ ]:
# pip install nbformat>=4.2.0

In [ ]:
# Visualize the topic-document distribution for a single document
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)

In [ ]:
topic_model.visualize_documents(data, embeddings=embeddings)

In [ ]:
topic_model.visualize_documents(data, reduced_embeddings=reduced_embeddings)

In [ ]:
topa_df = topic_model.get_document_info(src)

In [ ]:
topa_df

In [ ]:
topa_org_df = bert_src_af.iloc[:1000, :]
topa_org_df.reset_index(drop=True, inplace=True)
topa_df.reset_index(drop=True, inplace=True)

In [ ]:
tot_topa_df = pd.concat([topa_org_df, topa_df], axis = 1)

In [ ]:
tot_topa_df[tot_topa_df['Topic'] == 1]

In [ ]:
tot_topa_df[tot_topa_df['Topic'] == 2]